In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

from ram.analysis.run_manager import RunManager
from ram.analysis.model_selection.combo_search import CombinationSearch
from ram.analysis.statistics import get_stats, rollup_returns

In [2]:
RunManager.get_run_names('StatArbStrategy')

,RunName,RunDate,Completed,Description
0,run_0068,2018-03-06,True,"Sector 20, version 002"
1,run_0069,2018-03-07,True,"Sector 20, version 2 rerun with univ filtering"
2,run_0070,2018-03-19,True,"Sector 20, post refactor run"
3,run_0071,2018-03-21,True,"Sector 20, 2001 start date, 16cpu comp"
4,run_0072,2018-03-21,True,"Sector 20, 2001 start date, 64cpu comp"
5,run_0078,2018-04-18,True,"Sector 20, new pricing data"
6,run_0079,2018-04-25,True,Rerun with data since 2004
7,run_0080,2018-04-26,False,Rerun of version 24 data
8,run_0081,2018-04-30,True,New hyperparams
9,run_0085,2018-05-01,True,"new params, longer params/hyperparms"


In [3]:
run1 = RunManager('StatArbStrategy', 'run_0090', test_periods=5)
run2 = RunManager('StatArbStrategy', 'run_0092', test_periods=5)

In [4]:
run1.analyze_parameters()

,Param,Val,Count,MeanTotalRet,MeanSharpe,no_stat
0,holding_period,3,312,0.934319,0.064434,-999.0
1,holding_period,5,312,0.931341,0.066992,-999.0
2,holding_period,7,312,0.884437,0.065971,-999.0
3,model,"{u'max_features': 0.8, u'n_estimators': 30, u'...",468,1.002385,0.073161,-999.0
4,model,{u'type': u'reg'},468,0.831013,0.058437,-999.0
5,per_side_count,10,312,0.946631,0.068797,-999.0
6,per_side_count,20,312,0.736127,0.069799,-999.0
7,per_side_count,5,312,1.067338,0.058800,-999.0
8,response_days,10,468,0.912745,0.065360,-999.0
9,response_days,5,468,0.920652,0.066238,-999.0


In [5]:
run2.analyze_parameters()

,Param,Val,Count,MeanTotalRet,MeanSharpe,no_stat
0,holding_period,3,312,0.533284,0.036714,-999.0
1,holding_period,5,312,0.374762,0.027353,-999.0
2,holding_period,7,312,0.280471,0.020916,-999.0
3,model,"{u'max_features': 0.8, u'n_estimators': 30, u'...",468,0.551801,0.039419,-999.0
4,model,{u'type': u'reg'},468,0.240543,0.017237,-999.0
5,per_side_count,10,312,0.408156,0.029022,-999.0
6,per_side_count,20,312,0.346545,0.032860,-999.0
7,per_side_count,5,312,0.433816,0.023102,-999.0
8,response_days,10,468,0.412847,0.029659,-999.0
9,response_days,5,468,0.379497,0.026997,-999.0
